## Monitoring

## Clone Cisco Kubeflow Starter pack repository

In [1]:
BRANCH_NAME="dev" #Provide git branch name "master" or "dev"
! git clone -b $BRANCH_NAME https://github.com/CiscoAI/cisco-kubeflow-starter-pack.git

Cloning into 'cisco-kubeflow-starter-pack'...
remote: Enumerating objects: 311, done.
remote: Counting objects: 100% (311/311), done.
remote: Compressing objects: 100% (205/205), done.
remote: Total 4291 (delta 88), reused 246 (delta 58), pack-reused 3980
Receiving objects: 100% (4291/4291), 16.33 MiB | 26.46 MiB/s, done.
Resolving deltas: 100% (1592/1592), done.


In [2]:
! pip install pandas sklearn prometheus_client kfserving boto3 --user

     |████████████████████████████████| 10.1 MB 7.8 MB/s eta 0:00:01
     |████████████████████████████████| 83 kB 4.9 MB/s  eta 0:00:01
     |████████████████████████████████| 128 kB 50.8 MB/s eta 0:00:01
     |████████████████████████████████| 6.8 MB 45.4 MB/s eta 0:00:01
     |████████████████████████████████| 75 kB 12.0 MB/s eta 0:00:01
     |████████████████████████████████| 88 kB 14.2 MB/s eta 0:00:01
     |████████████████████████████████| 55 kB 15.8 MB/s eta 0:00:01
     |████████████████████████████████| 6.3 MB 133.1 MB/s eta 0:00:01
     |████████████████████████████████| 69 kB 28.2 MB/s eta 0:00:01
     |████████████████████████████████| 298 kB 146.0 MB/s eta 0:00:01
     |████████████████████████████████| 47 kB 13.3 MB/s eta 0:00:01
     |████████████████████████████████| 547 kB 138.0 MB/s eta 0:00:01
  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=2397 sha256=a1b9fd733b88a0ced24af1c6402448c41b981aea014dd6e7a591c7091dbb609d
  Stored in directory:

## Restart Notebook kernel

In [ ]:
from IPython.display import display_html
display_html("<script>Jupyter.notebook.kernel.restart()</script>",raw=True)

## Import Libraries

In [1]:
from __future__ import division
from __future__ import print_function

import tensorflow as tf
import pandas as pd
import numpy as np
import shutil
import yaml
import os
import logging
import requests
import json
from sklearn.preprocessing import OneHotEncoder

from kubernetes.client import V1Container
from kfserving import KFServingClient
from kfserving import constants
from kfserving import utils
from kfserving import V1alpha2EndpointSpec
from kfserving import V1alpha2PredictorSpec
from kfserving import V1alpha2InferenceServiceSpec
from kfserving import V1alpha2InferenceService
from kfserving import V1alpha2CustomSpec
from kfserving import V1alpha2Logger

from kubernetes import client as k8s_client
from kubernetes import config as k8s_config
from kubernetes.client.rest import ApiException
import boto3
from botocore.client import Config

k8s_config.load_incluster_config()
api_client = k8s_client.CoreV1Api()
custom_api=k8s_client.CustomObjectsApi()
namespace = utils.get_default_target_namespace()
print(namespace)

anonymous


## Prerequisites

Run the following command to install Prometheus and Grafana

In [2]:
!kubectl create namespace knative-monitoring
!kubectl apply --filename https://github.com/knative/serving/releases/download/v0.15.0/monitoring-metrics-prometheus.yaml

namespace/knative-monitoring created
serviceaccount/kube-state-metrics created
role.rbac.authorization.k8s.io/kube-state-metrics-resizer created
rolebinding.rbac.authorization.k8s.io/kube-state-metrics created
clusterrole.rbac.authorization.k8s.io/kube-state-metrics created
clusterrolebinding.rbac.authorization.k8s.io/kube-state-metrics created
deployment.apps/kube-state-metrics created
service/kube-state-metrics created
configmap/grafana-dashboard-definition-kubernetes-deployment created
configmap/grafana-dashboard-definition-kubernetes-capacity-planning created
configmap/grafana-dashboard-definition-kubernetes-cluster-health created
configmap/grafana-dashboard-definition-kubernetes-cluster-status created
configmap/grafana-dashboard-definition-kubernetes-control-plane-status created
configmap/grafana-dashboard-definition-kubernetes-resource-requests created
configmap/grafana-dashboard-definition-kubernetes-nodes created
configmap/grafana-dashboard-definition-kubernetes-pods created
co

Ensure that the grafana-*, kube-state-metrics-*, node-exporter-* and prometheus-system-* pods all report a Running status

In [3]:
!kubectl get pods --namespace knative-monitoring

NAME                                  READY   STATUS    RESTARTS   AGE
grafana-d6495dbb5-gxfqj               1/1     Running   0          38s
kube-state-metrics-5cb5c6986b-xc7xt   1/1     Running   0          39s
node-exporter-rtpr2                   2/2     Running   0          39s
prometheus-system-0                   1/1     Running   0          38s
prometheus-system-1                   0/1     Running   0          38s


## Deploy the Prometheus Pushgateway

In [4]:
def create_deployment_object():
    # Configureate Pod template container
    container = k8s_client.V1Container(
        name="pushgateway",
        image="prom/pushgateway",
        ports=[k8s_client.V1ContainerPort(container_port=9091)]
    )
    # Create and configurate a spec section
    template = k8s_client.V1PodTemplateSpec(
        metadata=k8s_client.V1ObjectMeta(labels={"app": "pushgateway"}),
        spec=k8s_client.V1PodSpec(containers=[container]))
    # Create the specification of deployment
    spec = k8s_client.V1DeploymentSpec(
        replicas=1,
        template=template,
        selector={'matchLabels': {'app': 'pushgateway'}})
    # Instantiate the deployment object
    deployment = k8s_client.V1Deployment(
        api_version="apps/v1",
        kind="Deployment",
        metadata=k8s_client.V1ObjectMeta(name="prometheus-pushgateway", 
                                         namespace=namespace,
                                        labels={'app': 'pushgateway'}),
        spec=spec)

    return deployment

def create_service_object():
    svc=k8s_client.V1Service(
            api_version="v1",
            kind="Service",
            metadata=k8s_client.V1ObjectMeta(
                name="prometheus-pushgateway-svc", 
                namespace=namespace,
                labels={"app":"pushgateway"},
                annotations={ 'prometheus.io/scrape': 'true',
                              'prometheus.io/port':   '9091'}
            ),
            spec=k8s_client.V1ServiceSpec(
                selector={"app":"pushgateway"},
                ports=[k8s_client.V1ServicePort(
                    name="pushgateway",
                    port=9091
                )]
            )
        )
    return svc

apps_v1 = k8s_client.AppsV1Api()
api_instance = k8s_client.CoreV1Api()
deploy=apps_v1.create_namespaced_deployment(namespace=namespace, body=create_deployment_object())
svc=api_instance.create_namespaced_service(namespace=namespace, body=create_service_object())

## Check GPUs availability

In [5]:
gpus = len(tf.config.experimental.list_physical_devices('GPU'))
if gpus == 0:
    print("Model will be trained using CPU")
elif gpus >= 0:
    print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
    tf.config.experimental.list_physical_devices('GPU')
    print("Model will be trained using GPU")

Model will be trained using CPU


## Declare Variables

In [6]:
path="cisco-kubeflow-starter-pack/apps/networking/ble-localization/onprem"
BLE_RSSI = pd.read_csv(os.path.join(path, "data/iBeacon_RSSI_Labeled.csv")) #Labeled dataset

# Configure model options
TF_DATA_DIR = os.getenv("TF_DATA_DIR", "/tmp/data/")
TF_MODEL_DIR = os.getenv("TF_MODEL_DIR", "blerssi/")
TF_EXPORT_DIR = os.getenv("TF_EXPORT_DIR", "blerssi/")
TF_MODEL_TYPE = os.getenv("TF_MODEL_TYPE", "DNN")
TF_TRAIN_STEPS = int(os.getenv("TF_TRAIN_STEPS", 5000))
TF_BATCH_SIZE = int(os.getenv("TF_BATCH_SIZE", 128))
TF_LEARNING_RATE = float(os.getenv("TF_LEARNING_RATE", 0.001))


# Feature columns
COLUMNS = list(BLE_RSSI.columns)
FEATURES = COLUMNS[2:]
def make_feature_cols():
  input_columns = [tf.feature_column.numeric_column(k) for k in FEATURES]
  return input_columns

## BLERSSI Input Dataset
### Attribute Information
location: The location of receiving RSSIs from ibeacons b3001 to b3013; 
          symbolic values showing the column and row of the location on the map (e.g., A01 stands for column A, row 1).
date: Datetime in the format of ‘d-m-yyyy hh:mm:ss’
b3001 - b3013: RSSI readings corresponding to the iBeacons; numeric, integers only.

In [7]:
BLE_RSSI.head(10)

,location,date,b3001,b3002,b3003,b3004,b3005,b3006,b3007,b3008,b3009,b3010,b3011,b3012,b3013
0,O02,10-18-2016 11:15:21,-200,-200,-200,-200,-200,-78,-200,-200,-200,-200,-200,-200,-200
1,P01,10-18-2016 11:15:19,-200,-200,-200,-200,-200,-78,-200,-200,-200,-200,-200,-200,-200
2,P01,10-18-2016 11:15:17,-200,-200,-200,-200,-200,-77,-200,-200,-200,-200,-200,-200,-200
3,P01,10-18-2016 11:15:15,-200,-200,-200,-200,-200,-77,-200,-200,-200,-200,-200,-200,-200
4,P01,10-18-2016 11:15:13,-200,-200,-200,-200,-200,-77,-200,-200,-200,-200,-200,-200,-200
5,P01,10-18-2016 11:15:11,-200,-200,-82,-200,-200,-200,-200,-200,-200,-200,-200,-200,-200
6,P01,10-18-2016 11:15:09,-200,-200,-80,-200,-200,-77,-200,-200,-200,-200,-200,-200,-200
7,P02,10-18-2016 11:15:07,-200,-200,-86,-200,-200,-200,-200,-200,-200,-200,-200,-200,-200
8,R01,10-18-2016 11:15:05,-200,-200,-200,-75,-200,-200,-200,-200,-200,-200,-200,-200,-200
9,R01,10-18-2016 11:15:03,-200,-200,-200,-75,-200,-200,-200,-200,-200,-200,-200,-200,-200


## Definition of Serving Input Receiver Function

In [8]:
feature_columns =  make_feature_cols()
inputs = {}
for feat in feature_columns:
  inputs[feat.name] = tf.placeholder(shape=[None], dtype=feat.dtype)
serving_input_receiver_fn = tf.estimator.export.build_raw_serving_input_receiver_fn(inputs)

## Train and Save BLE RSSI Model

In [9]:
# Feature columns
COLUMNS = list(BLE_RSSI.columns)
FEATURES = COLUMNS[2:]
LABEL = [COLUMNS[0]]

b3001 = tf.feature_column.numeric_column(key='b3001',dtype=tf.float64)
b3002 = tf.feature_column.numeric_column(key='b3002',dtype=tf.float64)
b3003 = tf.feature_column.numeric_column(key='b3003',dtype=tf.float64)
b3004 = tf.feature_column.numeric_column(key='b3004',dtype=tf.float64)
b3005 = tf.feature_column.numeric_column(key='b3005',dtype=tf.float64)
b3006 = tf.feature_column.numeric_column(key='b3006',dtype=tf.float64)
b3007 = tf.feature_column.numeric_column(key='b3007',dtype=tf.float64)
b3008 = tf.feature_column.numeric_column(key='b3008',dtype=tf.float64)
b3009 = tf.feature_column.numeric_column(key='b3009',dtype=tf.float64)
b3010 = tf.feature_column.numeric_column(key='b3010',dtype=tf.float64)
b3011 = tf.feature_column.numeric_column(key='b3011',dtype=tf.float64)
b3012 = tf.feature_column.numeric_column(key='b3012',dtype=tf.float64)
b3013 = tf.feature_column.numeric_column(key='b3013',dtype=tf.float64)
feature_columns = [b3001, b3002, b3003, b3004, b3005, b3006, b3007, b3008, b3009, b3010, b3011, b3012, b3013]

df_full = pd.read_csv(os.path.join(path, "data/iBeacon_RSSI_Labeled.csv")) #Labeled dataset

# Input Data Preprocessing 
df_full = df_full.drop(['date'],axis = 1)
df_full[FEATURES] = (df_full[FEATURES])/(-200)


#Output Data Preprocessing
dict = {'O02': 0,'P01': 1,'P02': 2,'R01': 3,'R02': 4,'S01': 5,'S02': 6,'T01': 7,'U02': 8,'U01': 9,'J03': 10,'K03': 11,'L03': 12,'M03': 13,'N03': 14,'O03': 15,'P03': 16,'Q03': 17,'R03': 18,'S03': 19,'T03': 20,'U03': 21,'U04': 22,'T04': 23,'S04': 24,'R04': 25,'Q04': 26,'P04': 27,'O04': 28,'N04': 29,'M04': 30,'L04': 31,'K04': 32,'J04': 33,'I04': 34,'I05': 35,'J05': 36,'K05': 37,'L05': 38,'M05': 39,'N05': 40,'O05': 41,'P05': 42,'Q05': 43,'R05': 44,'S05': 45,'T05': 46,'U05': 47,'S06': 48,'R06': 49,'Q06': 50,'P06': 51,'O06': 52,'N06': 53,'M06': 54,'L06': 55,'K06': 56,'J06': 57,'I06': 58,'F08': 59,'J02': 60,'J07': 61,'I07': 62,'I10': 63,'J10': 64,'D15': 65,'E15': 66,'G15': 67,'J15': 68,'L15': 69,'R15': 70,'T15': 71,'W15': 72,'I08': 73,'I03': 74,'J08': 75,'I01': 76,'I02': 77,'J01': 78,'K01': 79,'K02': 80,'L01': 81,'L02': 82,'M01': 83,'M02': 84,'N01': 85,'N02': 86,'O01': 87,'I09': 88,'D14': 89,'D13': 90,'K07': 91,'K08': 92,'N15': 93,'P15': 94,'I15': 95,'S15': 96,'U15': 97,'V15': 98,'S07': 99,'S08': 100,'L09': 101,'L08': 102,'Q02': 103,'Q01': 104}
df_full['location'] = df_full['location'].map(dict)
df_train=df_full.sample(frac=0.8,random_state=200)
df_valid=df_full.drop(df_train.index)

location_counts = BLE_RSSI.location.value_counts()
x1 = np.asarray(df_train[FEATURES])
y1 = np.asarray(df_train['location'])

x2 = np.asarray(df_valid[FEATURES])
y2 = np.asarray(df_valid['location'])

def formatFeatures(features):
    formattedFeatures = {}
    numColumns = features.shape[1]

    for i in range(0, numColumns):
        formattedFeatures["b"+str(3001+i)] = features[:, i]

    return formattedFeatures

trainingFeatures = formatFeatures(x1)
trainingCategories = y1

testFeatures = formatFeatures(x2)
testCategories = y2

# Train Input Function
def train_input_fn():
    dataset = tf.data.Dataset.from_tensor_slices((trainingFeatures, y1))
    dataset = dataset.repeat(1000).batch(TF_BATCH_SIZE)
    return dataset

# Test Input Function
def eval_input_fn():
    dataset = tf.data.Dataset.from_tensor_slices((testFeatures, y2))
    return dataset.repeat(1000).batch(TF_BATCH_SIZE)

# Provide list of GPUs should be used to train the model

distribution=tf.distribute.experimental.ParameterServerStrategy()
print('Number of devices: {}'.format(distribution.num_replicas_in_sync))

# Configuration of  training model

config = tf.estimator.RunConfig(train_distribute=distribution, model_dir=TF_MODEL_DIR, save_summary_steps=100, save_checkpoints_steps=100)

# Build 3 layer DNN classifier

model = tf.estimator.DNNClassifier(hidden_units = [13,65,110],
                 feature_columns = feature_columns,
                 model_dir = TF_MODEL_DIR,
                 n_classes=105, config=config
               )

export_final = tf.estimator.FinalExporter(TF_EXPORT_DIR, serving_input_receiver_fn=serving_input_receiver_fn)

train_spec = tf.estimator.TrainSpec(input_fn=train_input_fn, 
                                    max_steps=TF_TRAIN_STEPS)

eval_spec = tf.estimator.EvalSpec(input_fn=eval_input_fn,
                                  steps=100,
                                  exporters=export_final,
                                  throttle_secs=1,
                                  start_delay_secs=1)

# Train and Evaluate the model

tf.estimator.train_and_evaluate(model, train_spec, eval_spec)

INFO:tensorflow:ParameterServerStrategy with compute_devices = ('/device:CPU:0',), variable_device = '/device:CPU:0'


[I 200629 08:14:02 parameter_server_strategy:294] ParameterServerStrategy with compute_devices = ('/device:CPU:0',), variable_device = '/device:CPU:0'


Number of devices: 1
INFO:tensorflow:Initializing RunConfig with distribution strategies.


[I 200629 08:14:02 run_config:566] Initializing RunConfig with distribution strategies.


INFO:tensorflow:Not using Distribute Coordinator.


...

INFO:tensorflow:SavedModel written to: blerssi/export/blerssi/temp-b'1593418510'/saved_model.pb


[I 200629 08:15:10 builder_impl:425] SavedModel written to: blerssi/export/blerssi/temp-b'1593418510'/saved_model.pb


INFO:tensorflow:Loss for final step: 260.7274.


[I 200629 08:15:10 estimator:371] Loss for final step: 260.7274.


({'accuracy': 0.24632813,
  'average_loss': 3.0482693,
  'loss': 390.17847,
  'global_step': 5000},
 [b'blerssi/export/blerssi/1593418510'])

## Get pushgateway-service cluster IP to push metrics

In [10]:
try:
    pushgateway_svc_endpoint = api_client.read_namespaced_service(name=svc.metadata.name, namespace=namespace).spec.cluster_ip
    pushgateway_port = api_client.read_namespaced_service(name=svc.metadata.name, namespace=namespace).spec.ports[0].port
    minio_cluster_ip=api_client.read_namespaced_service(name="minio-service", namespace="kubeflow").spec.cluster_ip
    minio_cluster_port=api_client.read_namespaced_service(name="minio-service", namespace="kubeflow").spec.ports[0].port
except ApiException as e:
    if e.status == 403:
        logging.warning(f"The service account doesn't have sufficient privileges "
                      f"to get the pushgateway service. "
                      f"You will have to manually enter the pushgateway cluster-ip. "
                      f"To make this function work ask someone with cluster "
                      f"priveleges to create an appropriate "
                      f"clusterrolebinding by running a command.\n")
        logging.error("API access denied with reason: {e.reason}")
pushgateway_endpoint="http://"+pushgateway_svc_endpoint+":%s"%pushgateway_port
minio_endpoint="http://"+minio_cluster_ip+":%s"%minio_cluster_port
print(minio_endpoint)
print(pushgateway_endpoint)

http://10.96.231.123:9000
http://10.107.178.115:9091


## Inference Logger
Create a message dumper knative service to print out CloudEvents it receives

In [11]:
message_dumper=f"""apiVersion: serving.knative.dev/v1alpha1
kind: Service
metadata:
  name: message-dumper
  namespace: {namespace}
spec:
  template:
    spec:
      containers:
      - image: gcr.io/knative-releases/github.com/knative/eventing-sources/cmd/event_display
"""
spec=yaml.safe_load(message_dumper)

In [12]:
custom_api.create_namespaced_custom_object(group="serving.knative.dev", version="v1alpha1", namespace=namespace, plural="services", body=spec)

{'apiVersion': 'serving.knative.dev/v1alpha1',
 'kind': 'Service',
 'metadata': {'annotations': {'serving.knative.dev/creator': 'system:serviceaccount:anonymous:default-editor',
   'serving.knative.dev/lastModifier': 'system:serviceaccount:anonymous:default-editor'},
  'creationTimestamp': '2020-06-29T08:15:41Z',
  'generation': 1,
  'name': 'message-dumper',
  'namespace': 'anonymous',
  'resourceVersion': '1772085',
  'selfLink': '/apis/serving.knative.dev/v1alpha1/namespaces/anonymous/services/message-dumper',
  'uid': '526ebeb1-6f56-4502-b2a9-531159e0f9fe'},
 'spec': {'template': {'metadata': {'creationTimestamp': None},
   'spec': {'containerConcurrency': 0,
    'containers': [{'image': 'gcr.io/knative-releases/github.com/knative/eventing-sources/cmd/event_display',
      'name': 'user-container',
      'readinessProbe': {'successThreshold': 1, 'tcpSocket': {'port': 0}},
      'resources': {}}],
    'timeoutSeconds': 300}},
  'traffic': [{'latestRevision': True, 'percent': 100}]}}

## KFServing Client SDK

We will use the KFServing client SDK to create the InferenceService and deploy our custom image.

### Define InferenceService

Firstly define default endpoint spec, and then define the inferenceservice using the endpoint spec.

To use a custom image we need to use V1alphaCustomSpec which takes a V1Container from the kuberenetes library

Kfserving custom image Source Code https://github.com/CiscoAI/cisco-kubeflow-starter-pack/tree/dev/apps/networking/ble-localization/onprem/knative-monitoring/model-server

In [13]:
MODEL_NAME="blerssi-model"
BUCKET_NAME="kfserving"
pvcname = !(echo  $HOSTNAME | sed 's/.\{2\}$//')
pvc = "workspace-"+pvcname[0]
url="http://"+spec['metadata']['name']+'.'+spec['metadata']['namespace']+'/'
api_version = constants.KFSERVING_GROUP + '/' + constants.KFSERVING_VERSION
default_endpoint_spec = V1alpha2EndpointSpec(
                          predictor=V1alpha2PredictorSpec(
                              logger=V1alpha2Logger(mode="all", url=url), #Available modes request, response and all
                              custom=V1alpha2CustomSpec(
                                  container=V1Container(
                                      name="kfserving-container",
                                      image="docker.io/samba07/kfserving-custom-model:0.3",
                                      env=[{"name":"STORAGE_URI", "value":"pvc://%s"%pvc}],
                                      command=["python"],
                                      args=[
                                          "model.py",
                                          "--pushgateway", "%s"%pushgateway_endpoint,
                                          "--storage_uri", "pvc://%s/blerssi/export/blerssi"%pvc,
                                          "--out_dir", "/mnt/pvc/blerssi/export/blerssi",
                                          "bucket_name","%s"%BUCKET_NAME
                                          ]))))

isvc = V1alpha2InferenceService(api_version=api_version,
                          kind=constants.KFSERVING_KIND,
                          metadata=k8s_client.V1ObjectMeta(
                              annotations={"prometheus.io/scrape": 'true',
                                           "prometheus.io/port":   '9091'},
                              name=MODEL_NAME, namespace=namespace),
                                spec=V1alpha2InferenceServiceSpec(default=default_endpoint_spec)
                                )

## Create the InferenceService
Call KFServingClient to create InferenceService.

In [14]:
KFServing = KFServingClient()
KFServing.create(isvc)

{'apiVersion': 'serving.kubeflow.org/v1alpha2',
 'kind': 'InferenceService',
 'metadata': {'annotations': {'prometheus.io/port': '9091',
   'prometheus.io/scrape': 'true'},
  'creationTimestamp': '2020-06-29T08:16:05Z',
  'generation': 1,
  'name': 'blerssi-model',
  'namespace': 'anonymous',
  'resourceVersion': '1772295',
  'selfLink': '/apis/serving.kubeflow.org/v1alpha2/namespaces/anonymous/inferenceservices/blerssi-model',
  'uid': 'f585751b-50a6-424a-813e-eef736393f84'},
 'spec': {'default': {'predictor': {'custom': {'container': {'args': ['model.py',
       '--pushgateway',
       'http://10.107.178.115:9091',
       '--storage_uri',
       'pvc://workspace-ravi/blerssi/export/blerssi',
       '--out_dir',
       '/mnt/pvc/blerssi/export/blerssi',
       'bucket_name',
       'kfserving'],
      'command': ['python'],
      'env': [{'name': 'STORAGE_URI', 'value': 'pvc://workspace-ravi'}],
      'image': 'docker.io/samba07/kfserving-custom-model:0.3',
      'name': 'kfserving-co

## Check the InferenceService

In [15]:
KFServing.get(MODEL_NAME, namespace=namespace, watch=True)

NAME                 READY      DEFAULT_TRAFFIC CANARY_TRAFFIC  URL                                               
blerssi-model        False                                                                                        
blerssi-model        False                                                                                        
blerssi-model        True       100                             http://blerssi-model.anonymous.example.com/v1/m...


## Run a prediction

In [16]:
CLUSTER='ucs' #where your cluster running 'gcp' or 'ucs'

In [17]:
%%bash -s "$CLUSTER" --out NODE_IP
if [ $1 = "ucs" ]
then
    echo "$(kubectl get node -o=jsonpath='{.items[0].status.addresses[0].address}')"
else
    echo "$(kubectl get node -o=jsonpath='{.items[0].status.addresses[1].address}')"
fi

In [18]:
%%bash --out INGRESS_PORT
INGRESS_GATEWAY="istio-ingressgateway"
echo "$(kubectl -n istio-system get service $INGRESS_GATEWAY -o jsonpath='{.spec.ports[1].nodePort}')"

In [19]:
%%bash -s "$MODEL_NAME" --out SERVICE_HOSTNAME
echo "$(kubectl get inferenceservice $1 -o jsonpath='{.status.url}' | cut -d "/" -f 3)"

### Data for prediction

In [20]:
path="cisco-kubeflow-starter-pack/apps/networking/ble-localization/onprem"
df_full = pd.read_csv(os.path.join(path,'data/iBeacon_RSSI_Unlabeled_truncated.csv')) #Labeled dataset
  # Input Data Preprocessing 
df_full = df_full.drop(['date'],axis = 1)
df_full = df_full.drop(['location'],axis = 1)
input_data=df_full.to_numpy()[:105]

In [21]:
url = f"http://{NODE_IP.strip()}:{INGRESS_PORT.strip()}/v1/models/{MODEL_NAME}:predict"
headers = {"Host": SERVICE_HOSTNAME.strip()}
if not len(input_data)==13 and not type(input_data[0]) ==list:
    for row in input_data.tolist():
        data={"instances": row}
        response=requests.post(url, data=json.dumps(data), headers=headers)
        predictions=response.json()['class_ids']
        print(predictions)
else:
    data={"instances":input_data.tolist()}
    response=requests.post(url, data=json.dumps(data), headers=headers)
    predictions=response.json()['class_ids']
    print(predictions)

[[94]]
[[94]]
[[94]]
[[94]]
[[94]]
[[94]]
[[94]]
[[94]]
[[94]]
[[94]]
[[94]]
[[94]]
[[94]]
[[94]]
[[94]]
[[94]]
[[94]]
[[94]]
[[2]]
[[2]]
[[2]]
[[2]]
[[2]]
[[94]]
[[94]]
[[94]]
[[94]]
[[94]]
[[94]]
[[94]]
[[94]]
[[94]]
[[94]]
[[94]]
[[94]]
[[94]]
[[94]]
[[94]]
[[94]]
[[94]]
[[94]]
[[94]]
[[94]]
[[94]]
[[94]]
[[94]]
[[94]]
[[94]]
[[94]]
[[94]]
[[94]]
[[94]]
[[94]]
[[94]]
[[94]]
[[94]]
[[94]]
[[94]]
[[94]]
[[94]]
[[94]]
[[94]]
[[94]]
[[94]]
[[94]]
[[94]]
[[94]]
[[94]]
[[94]]
[[94]]
[[94]]
[[94]]
[[94]]
[[94]]
[[94]]
[[94]]
[[94]]
[[94]]
[[94]]
[[94]]
[[94]]
[[94]]
[[94]]
[[94]]
[[94]]
[[94]]
[[94]]
[[94]]
[[94]]
[[94]]
[[94]]
[[94]]
[[94]]
[[94]]
[[94]]
[[94]]
[[94]]
[[94]]
[[94]]
[[94]]
[[94]]
[[94]]
[[94]]
[[94]]
[[94]]


In [26]:
%%bash -s "$namespace" --out message_dumper

echo "$(kubectl get pod -n $1 -l serving.knative.dev/service=message-dumper -o jsonpath='{.items[0].metadata.name}')"

In [30]:
message_dumper=message_dumper.strip()

In [31]:
!kubectl logs  $message_dumper user-container -n $namespace --tail 153

☁️  cloudevents.Event
Validation: valid
Context Attributes,
  specversion: 0.2
  type: org.kubeflow.serving.inference.request
  source: http://localhost:8080/
  id: 41abfec8-a782-4e1b-be3e-3ce73250ce86
  time: 2020-06-29T08:29:31.189612269Z
  contenttype: application/json
Data,
  {
    "instances": [
      -200,
      -200,
      -80,
      -200,
      -77,
      -56,
      -81,
      -200,
      -200,
      -200,
      -200,
      -200,
      -200
    ]
  }
☁️  cloudevents.Event
Validation: valid
Context Attributes,
  specversion: 0.2
  type: org.kubeflow.serving.inference.response
  source: http://localhost:8080/
  id: 41abfec8-a782-4e1b-be3e-3ce73250ce86
  time: 2020-06-29T08:29:32.531109312Z
  contenttype: application/json
Data,
  {
    "probabilities": [
      [
        8.361453365068883e-05,
        3.7587375118164346e-05,
        0.0391666404902935,
        0.005307038314640522,
        1.4862819170957664e-06,
        0.0010913386940956116,
        0.0004093000025022775,
       

## Clean Up

### Delete the InferenceService

In [32]:
KFServing.delete(MODEL_NAME, namespace=namespace)

{'kind': 'Status',
 'apiVersion': 'v1',
 'metadata': {},
 'status': 'Success',
 'details': {'name': 'blerssi-model',
  'group': 'serving.kubeflow.org',
  'kind': 'inferenceservices',
  'uid': 'f585751b-50a6-424a-813e-eef736393f84'}}

## Delete message dumper

In [33]:
custom_api.delete_namespaced_custom_object(group="serving.knative.dev", version="v1alpha1", name=spec['metadata']['name'],namespace=namespace, plural="services", body=k8s_client.V1DeleteOptions())

{'kind': 'Status',
 'apiVersion': 'v1',
 'metadata': {},
 'status': 'Success',
 'details': {'name': 'message-dumper',
  'group': 'serving.knative.dev',
  'kind': 'services',
  'uid': '526ebeb1-6f56-4502-b2a9-531159e0f9fe'}}

### Delete the Prometheus pushgateway deployment and service

In [34]:
api_instance.delete_namespaced_service(name=svc.metadata.name, namespace=namespace)

{'api_version': 'v1',
 'code': None,
 'details': {'causes': None,
             'group': None,
             'kind': 'services',
             'name': 'prometheus-pushgateway-svc',
             'retry_after_seconds': None,
             'uid': '22b25e6b-ba5a-4973-8a04-ffe638ad138e'},
 'kind': 'Status',
 'message': None,
 'metadata': {'_continue': None, 'resource_version': None, 'self_link': None},
 'reason': None,
 'status': 'Success'}

In [35]:
apps_v1.delete_namespaced_deployment(name=deploy.metadata.name, namespace=namespace)

{'api_version': 'v1',
 'code': None,
 'details': {'causes': None,
             'group': 'apps',
             'kind': 'deployments',
             'name': 'prometheus-pushgateway',
             'retry_after_seconds': None,
             'uid': '6f51610b-d8b7-40a1-878f-b3178ca262e4'},
 'kind': 'Status',
 'message': None,
 'metadata': {'_continue': None, 'resource_version': None, 'self_link': None},
 'reason': None,
 'status': 'Success'}

### Uninstall Prometheus and Grafana

In [36]:
!kubectl delete --filename https://github.com/knative/serving/releases/download/v0.15.0/monitoring-metrics-prometheus.yaml
!kubectl delete namespace knative-monitoring

serviceaccount "kube-state-metrics" deleted
role.rbac.authorization.k8s.io "kube-state-metrics-resizer" deleted
rolebinding.rbac.authorization.k8s.io "kube-state-metrics" deleted
clusterrole.rbac.authorization.k8s.io "kube-state-metrics" deleted
clusterrolebinding.rbac.authorization.k8s.io "kube-state-metrics" deleted
deployment.apps "kube-state-metrics" deleted
service "kube-state-metrics" deleted
configmap "grafana-dashboard-definition-kubernetes-deployment" deleted
configmap "grafana-dashboard-definition-kubernetes-capacity-planning" deleted
configmap "grafana-dashboard-definition-kubernetes-cluster-health" deleted
configmap "grafana-dashboard-definition-kubernetes-cluster-status" deleted
configmap "grafana-dashboard-definition-kubernetes-control-plane-status" deleted
configmap "grafana-dashboard-definition-kubernetes-resource-requests" deleted
configmap "grafana-dashboard-definition-kubernetes-nodes" deleted
configmap "grafana-dashboard-definition-kubernetes-pods" deleted
configmap

### Delete minio bucket

In [37]:
boto_client = boto3.client('s3', endpoint_url=minio_endpoint, aws_access_key_id="minio",aws_secret_access_key="minio123",
                      config=Config(signature_version='s3v4'),region_name="us-east-1", use_ssl=False)
boto_client.delete_object(Bucket=BUCKET_NAME, Key="kfserving_metrics.csv")
boto_client.delete_bucket(Bucket=BUCKET_NAME)

{'ResponseMetadata': {'RequestId': '161CF6A0BB923880',
  'HostId': '',
  'HTTPStatusCode': 204,
  'HTTPHeaders': {'accept-ranges': 'bytes',
   'server': 'Minio/RELEASE.2018-02-09T22-40-05Z (linux; amd64)',
   'vary': 'Origin',
   'x-amz-request-id': '161CF6A0BB923880',
   'date': 'Mon, 29 Jun 2020 08:32:18 GMT'},
  'RetryAttempts': 0}}